In [ ]:
# Set up

import pandas as pd
import numpy as np
import os
import gc
from tqdm.notebook import tqdm
import datetime as dt

print(os.getcwd)
os.chdir("..")
script_path = os.getcwd()
data_path = os.path.join(script_path, "data")


In [ ]:
# 1. Before processing the MOT Data, read in the vehicle attribute databases and produce the RFR models for CO2 and mass

# Read in databases
# 1.1 BEV data for EL
# from EV Database https://ev-database.org/uk/
fpath = os.path.join(data_path, "ev_data", "bev_spec.csv")
bev_spec = pd.read_csv(fpath)

# only keep cols we need:
bev_spec = bev_spec[['first_use_year', 'make', 'model', "fuel efficiency Wh/mi", "battery capacity (kWh)", "mass (kg)"]]
# drop duplicates and keep first in index order
bev_spec = bev_spec.drop_duplicates(keep='first')

# 1.2 Read in the VCA data for PHEVs
fpath = os.path.join(data_path, "vca_data", "vca_phev_lookup.csv")
vca_phev_lookup = pd.read_csv(fpath)
# Modify the vca_phev_lookup data to match the MOT data
vca_phev_lookup["fuel_type"] = vca_phev_lookup["Fuel Type"]
vca_phev_lookup.loc[vca_phev_lookup["fuel_type"] == "Petrol", "fuel_type"] = "PE"
vca_phev_lookup.loc[vca_phev_lookup["fuel_type"] == "Petrol Electric", "fuel_type"] = "HY"
vca_phev_lookup.loc[vca_phev_lookup["fuel_type"] == "Electricity / Petrol", "fuel_type"] = "HY"
vca_phev_lookup.loc[vca_phev_lookup["fuel_type"] == "Petrol Hybrid", "fuel_type"] = "HY"      
vca_phev_lookup.loc[vca_phev_lookup["fuel_type"] == "Electricity", "fuel_type"] = "EL"
vca_phev_lookup.loc[vca_phev_lookup["fuel_type"] == "Diesel", "fuel_type"] = "DI"
vca_phev_lookup.loc[vca_phev_lookup["fuel_type"] == "Diesel Electric", "fuel_type"] = "HY"
vca_phev_lookup.loc[vca_phev_lookup["fuel_type"] == "Diesel Hybrid", "fuel_type"] = "HY"
vca_phev_lookup.loc[vca_phev_lookup["fuel_type"] == "Electricity / Diesel", "fuel_type"] = "HY"
vca_phev_lookup = vca_phev_lookup.drop_duplicates()

# 1.3 VCA Database RFR Model for CO2 - for DI PE HY
# From VCA https://carfueldata.vehicle-certification-agency.gov.uk/
data = []
for year in range(2001, 2022, 1):
    fname = os.path.join(data_path, "vca_data", f"vca_{year}.csv")
    data_year = pd.read_csv(fname, quoting=1, encoding='ISO-8859-1')
    data_year['Year'] = year
    data.append(data_year)
data = pd.concat(data, ignore_index=True)
# Process the VCA data
# Convert the numeric
data["Engine Capacity"] = pd.to_numeric(data["Engine Capacity"], errors='coerce')
data["CO2 g/km"] = pd.to_numeric(data["CO2 g/km"], errors='coerce')
# Modify the data data to match the MOT data
data["fuel_type"] = data["Fuel Type"]
data.loc[data["fuel_type"] == "Petrol", "fuel_type"] = "PE"
data.loc[data["fuel_type"] == "Petrol Electric", "fuel_type"] = "HY"
data.loc[data["fuel_type"] == "Electricity / Petrol", "fuel_type"] = "HY"
data.loc[data["fuel_type"] == "Petrol Hybrid", "fuel_type"] = "HY"      
data.loc[data["fuel_type"] == "Electricity", "fuel_type"] = "EL"
data.loc[data["fuel_type"] == "Diesel", "fuel_type"] = "DI"
data.loc[data["fuel_type"] == "Diesel Electric", "fuel_type"] = "HY"
data.loc[data["fuel_type"] == "Diesel Hybrid", "fuel_type"] = "HY"
data.loc[data["fuel_type"] == "Electricity / Diesel", "fuel_type"] = "HY"
# remove EL
data = data[data["fuel_type"] != "EL"]
data = data[data["fuel_type"].isin(["DI", "PE", "HY"])]

# Before calculating emissions, we need to adjust the CO2
# emissions for NEDC & WLTP to be reaL world emissions
# using the correction factors from the icct papers
adjustment_factors = {
    2000 : { "DI": 1.08, "PE": 1.04, "HY": 1.20,}, # NEDC
    2001 : { "DI": 1.08, "PE": 1.04, "HY": 1.20,}, # NEDC
    2002 : { "DI": 1.10, "PE": 1.05, "HY": 1.20,}, # NEDC
    2003 : { "DI": 1.10, "PE": 1.06, "HY": 1.20,}, # NEDC
    2004 : { "DI": 1.11, "PE": 1.07, "HY": 1.20,}, # NEDC
    2005 : { "DI": 1.13, "PE": 1.09, "HY": 1.20,}, # NEDC
    2006 : { "DI": 1.12, "PE": 1.10, "HY": 1.19,}, # NEDC
    2007 : { "DI": 1.14, "PE": 1.12, "HY": 1.19,}, # NEDC
    2008 : { "DI": 1.17, "PE": 1.14, "HY": 1.19,}, # NEDC
    2009 : { "DI": 1.18, "PE": 1.17, "HY": 1.29,}, # NEDC
    2010 : { "DI": 1.22, "PE": 1.19, "HY": 1.31,}, # NEDC
    2011 : { "DI": 1.26, "PE": 1.22, "HY": 1.36,}, # NEDC
    2012 : { "DI": 1.29, "PE": 1.26, "HY": 1.41,}, # NEDC
    2013 : { "DI": 1.34, "PE": 1.30, "HY": 1.44,}, # NEDC
    2014 : { "DI": 1.38, "PE": 1.32, "HY": 1.46,}, # NEDC
    2015 : { "DI": 1.43, "PE": 1.35, "HY": 1.49,}, # NEDC
    2016 : { "DI": 1.45, "PE": 1.36, "HY": 1.53,}, # NEDC
    2017 : { "DI": 1.45, "PE": 1.37, "HY": 1.51,}, # NEDC
    2018 : { "DI": 1.44, "PE": 1.37, "HY": 1.50,}, # NEDC
    2019 : { "DI": 1.07, "PE": 1.09, "HY": 1.09,}, # WLTP note HEVs use petrol correction factor for WLTP (ICCT do not provide correction factors for HEVs)
    2020 : { "DI": 1.10, "PE": 1.11, "HY": 1.11,}, # WLTP note HEVs use petrol correction factor for WLTP (ICCT do not provide correction factors for HEVs)
    2021 : { "DI": 1.10, "PE": 1.11, "HY": 1.11,}, # WLTP note HEVs use petrol correction factor for WLTP (ICCT do not provide correction factors for HEVs)
}
for year in range(2001, 2022, 1):
    for fuel in ["DI", "PE", "HY"]:
        data.loc[(data["Year"] == year) & (data["fuel_type"] == fuel), "CO2 g/km"] = data.loc[(data["Year"] == year) & (data["fuel_type"] == fuel), "CO2 g/km"] * adjustment_factors[year][fuel]


# find the CO2 per km for each year - just for reference
for year in range(2001, 2022, 1):
    data_year = data[data["Year"] == year]
    mean_year = round(data_year['CO2 g/km'].mean())
    di_year = round(data_year.loc[data_year["fuel_type"] == "DI", "CO2 g/km"].mean())
    pe_year = round(data_year.loc[data_year["fuel_type"] == "PE", "CO2 g/km"].mean())
    try:
        hy_year = round(data_year.loc[data_year["fuel_type"] == "HY", "CO2 g/km"].mean())
    except:
        hy_year = 0
    print(f"Year: {year}, CO2 g/km: {mean_year}, DI: {di_year}, PE: {pe_year}, HY: {hy_year}")

# drop any rows with nans
data = data[["fuel_type", "Year", "Engine Capacity", "CO2 g/km",]].dropna()

X = data[["fuel_type", "Year", "Engine Capacity"]]
y = data["CO2 g/km"]

# Data Preprocessing
X = X.copy()
y = y.copy()
# Encode categorical variables
le = LabelEncoder()
X['fuel_type'] = le.fit_transform(X['fuel_type'])
# Store feature names
co2_feature_names = ["Fuel Type", "Year", "Engine Capacity"]
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("CO2 model data shape:", X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# Create and train the model
rf_model_co2 = RandomForestRegressor(
    n_estimators=200,
    max_depth=None,
    min_samples_split=25,
    min_samples_leaf=5,
    random_state=42,
    n_jobs=-1
)
rf_model_co2.fit(X, y)
rf_model_co2.fit(X_train, y_train)

# Calculate and print R-squared for CO2 model
co2_train_r2 = rf_model_co2.score(X_train, y_train)
co2_test_r2 = rf_model_co2.score(X_test, y_test)
print(f"\nCO2 Model R-squared - Training: {co2_train_r2:.4f}, Testing: {co2_test_r2:.4f}")

# Get and print feature importance for CO2 model
co2_importance = rf_model_co2.feature_importances_
print("\nCO2 Model Feature Importance:")
for name, imp in zip(co2_feature_names, co2_importance):
    print(f"{name}: {imp:.4f}")
print("\n")    

# Finally plot the two models predictons vs test
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

# CO2 model
y_pred = rf_model_co2.predict(X_test)
ax1.hexbin(y_test, y_pred, gridsize=50, cmap='Blues', extent=(0, 350, 0, 350))
ax1.set_xlim(0, 350)
ax1.set_ylim(0, 350)
ax1.set_ylabel("Predicted CO2 g/km")
ax1.set_xlabel("Actual CO2 g/km")

# 1.4 EEA Database RFR Model for Mass - for DI PE HY
# From EEA - data is in multiple formats
data = []
for year in range(2010, 2022, 1):
    fname = os.path.join(data_path, "eea_data", f"eea_{year}.csv")
    if year < 2017:
        data_year = pd.read_csv(fname, usecols=['MS', 'Ft', 'Ec (cm3)', 'Enedc (g/km)',  'M (kg)'], engine='pyarrow')
        data_year['Ewltp (g/km)'] = np.nan
        data_year['z (Wh/km)'] = np.nan
    elif year == 2017:
        data_year = pd.read_csv(fname, encoding="utf-16-le", usecols=['MS', 'Ft', 'ec (cm3)', 'Enedc (g/km)', 'Ewltp (g/km)',  'm (kg)', 'z (Wh/km)'], sep='\t', engine='pyarrow')
        data_year = data_year.rename(columns={'ec (cm3)': 'Ec (cm3)', 'm (kg)': 'M (kg)'})
    elif year == 2018:
        data_year = pd.read_csv(fname, encoding="utf-8", usecols=['MS', 'Ft', 'ec (cm3)', 'Enedc (g/km)', 'Ewltp (g/km)',  'm (kg)', 'z (Wh/km)'], sep='\t', engine='pyarrow')
        data_year = data_year.rename(columns={'ec (cm3)': 'Ec (cm3)', 'm (kg)': 'M (kg)'})
    elif year >= 2019:
        data_year = pd.read_csv(fname, encoding="utf-8", usecols=['Country', 'Ft', 'ec (cm3)', 'Enedc (g/km)', 'Ewltp (g/km)',  'm (kg)', 'z (Wh/km)'], sep=',', engine='pyarrow')
        data_year = data_year.rename(columns={'ec (cm3)': 'Ec (cm3)', 'm (kg)': 'M (kg)', 'Country': 'MS'})
        
    data_year['Year'] = year  
    data_year = data_year.rename(columns={'Ec (cm3)': 'Engine Capacity', 'Enedc (g/km)': 'NEDC CO2 g/km', 'Ewltp (g/km)': 'WLTP CO2 g/km', 'M (kg)': 'mass (kg)', 'Ft': 'fuel_type'})
    data_year = data_year.drop_duplicates()
    data.append(data_year)
data = pd.concat(data, ignore_index=True)

# Process the EEA data
# # Keep only NEDC up to 2018 and WLTP from 2019
data["CO2 g/km"] = data["NEDC CO2 g/km"]
data.loc[data["Year"] >= 2019, "CO2 g/km"] = data.loc[data["Year"] >= 2019, "WLTP CO2 g/km"]
data = data.drop(columns=['NEDC CO2 g/km', 'WLTP CO2 g/km'])
# Convert the numeric
data["Engine Capacity"] = pd.to_numeric(data["Engine Capacity"], errors='coerce')
data["CO2 g/km"] = pd.to_numeric(data["CO2 g/km"], errors='coerce')
data["mass (kg)"] = pd.to_numeric(data["mass (kg)"], errors='coerce')
data["Enedc (g/km)"] = pd.to_numeric(data["CO2 g/km"], errors='coerce')
data["Ewltp (g/km)"] = pd.to_numeric(data["CO2 g/km"], errors='coerce')
data["z (Wh/km)"] = pd.to_numeric(data["z (Wh/km)"], errors='coerce')
# Modify the data data to match the MOT data
data["fuel_type"] = data["fuel_type"]
data.loc[data["fuel_type"] == "Petrol", "fuel_type"] = "PE"
data.loc[data["fuel_type"] == "PETROL", "fuel_type"] = "PE"
data.loc[data["fuel_type"] == "petrol", "fuel_type"] = "PE"
data.loc[data["fuel_type"] == "Petrol/Electric", "fuel_type"] = "HY"
data.loc[data["fuel_type"] == "PETROL/ELECTRIC", "fuel_type"] = "HY"
data.loc[data["fuel_type"] == "petrol/electric", "fuel_type"] = "HY"
data.loc[data["fuel_type"] == "petrol-electric", "fuel_type"] = "HY"
data.loc[data["fuel_type"] == "Petrol-Electric", "fuel_type"] = "HY"
data.loc[data["fuel_type"] == "PETROL-ELECTRIC", "fuel_type"] = "HY"
data.loc[data["fuel_type"] == "Diesel", "fuel_type"] = "DI"
data.loc[data["fuel_type"] == "DIESEL", "fuel_type"] = "DI"
data.loc[data["fuel_type"] == "diesel", "fuel_type"] = "DI"
data.loc[data["fuel_type"] == "Diesel/Electric", "fuel_type"] = "HY"
data.loc[data["fuel_type"] == "DIESEL/ELECTRIC", "fuel_type"] = "HY"
data.loc[data["fuel_type"] == "diesel/electric", "fuel_type"] = "HY"
data.loc[data["fuel_type"] == "diesel-electric", "fuel_type"] = "HY"
data.loc[data["fuel_type"] == "Diesel-Electric", "fuel_type"] = "HY"
data.loc[data["fuel_type"] == "DIESEL-ELECTRIC", "fuel_type"] = "HY"
data.loc[data["fuel_type"] == "Electric", "fuel_type"] = "EL"
data.loc[data["fuel_type"] == "ELECTRIC", "fuel_type"] = "EL"
data.loc[data["fuel_type"] == "electric", "fuel_type"] = "EL"
# only keep DI PE and HY
data = data[data["fuel_type"].isin(["DI", "PE", "HY"])]
# Lastly, exclude any 'z (Wh/km)' values that are not 0 - excludes PHEVs
# fill nan as 0
data["z (Wh/km)"] = data["z (Wh/km)"].fillna(0)
data = data[data["z (Wh/km)"] == 0]
print(f"Length of data: {len(data)}")
data = data[['fuel_type', 'Year', 'Engine Capacity', 'CO2 g/km', 'mass (kg)']].dropna()
data = data.drop_duplicates()
print(f"Length of data: {len(data)}")

# Adjuts type approval to real world using the ICCT adjustment factors
for year in range(2010, 2022, 1):
    for fuel in ["DI", "PE", "HY"]:
        data.loc[(data["Year"] == year) & (data["fuel_type"] == fuel), "CO2 g/km"] = data.loc[(data["Year"] == year) & (data["fuel_type"] == fuel), "CO2 g/km"] * adjustment_factors[year][fuel]

# find the mass for each year
for year in range(2010, 2022, 1):
    data_year = data[data["Year"] == year]
    mean_year = round(data_year["mass (kg)"].mean())
    di_year = round(data_year.loc[data_year["fuel_type"] == "DI", "mass (kg)"].mean())
    pe_year = round(data_year.loc[data_year["fuel_type"] == "PE", "mass (kg)"].mean())
    try:
        hy_year = round(data_year.loc[data_year["fuel_type"] == "HY", "mass (kg)"].mean())
    except:
        hy_year = 0
    # print for refrence
    print(f"Year: {year}, mass (kg): {mean_year}, DI: {di_year}, PE: {pe_year}, HY: {hy_year}")
    
# drop any rows with nans
data = data[["fuel_type", "Year", "Engine Capacity", "CO2 g/km", "mass (kg)"]].dropna()

# Create RFR model to predict mass
X = data[["fuel_type", "Year", "Engine Capacity", "CO2 g/km"]]
y = data["mass (kg)"]
# Data Preprocessing
X = X.copy()
y = y.copy()
# Encode categorical variables
le = LabelEncoder()
X['fuel_type'] = le.fit_transform(X['fuel_type'])
# Store feature names
mass_feature_names = ["Fuel Type", "Year", "Engine Capacity", "CO2 g/km"]
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("Mass model data shape:", X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# Create and train the model
rf_model_mass = RandomForestRegressor(
    n_estimators=200,
    max_depth=None,
    min_samples_split=25,
    min_samples_leaf=5,
    random_state=42,
    n_jobs=-1
)

rf_model_mass.fit(X_train, y_train)

# Calculate and print R-squared for mass model
mass_train_r2 = rf_model_mass.score(X_train, y_train)
mass_test_r2 = rf_model_mass.score(X_test, y_test)
print(f"\nMass Model R-squared - Training: {mass_train_r2:.4f}, Testing: {mass_test_r2:.4f}")

# Get and print feature importance for mass model
mass_importance = rf_model_mass.feature_importances_
print("\nMass Model Feature Importance:")
for name, imp in zip(mass_feature_names, mass_importance):
    print(f"{name}: {imp:.4f}")
print("\n")     

# Mass model
X_test_mass = X_test
y_test_mass = y_test
y_pred_mass = rf_model_mass.predict(X_test_mass)
ax2.hexbin(y_test_mass, y_pred_mass, gridsize=50, cmap='Blues', extent=(0, 3000, 0, 3000))
ax2.set_xlim(0, 3000)
ax2.set_ylim(0, 3000)
ax2.set_ylabel("Predicted mass (kg)")
ax2.set_xlabel("Actual mass (kg)")

plt.show()


# Fianlly repeat predictions for CO2 using the eea data
# drop any rows with nans
data = data[["fuel_type", "Year", "Engine Capacity", "CO2 g/km"]].dropna()
# Create RFR model to predict mass
X = data[["fuel_type", "Year", "Engine Capacity",]]
y = data[ "CO2 g/km"]
# Data Preprocessing
X = X.copy()
y = y.copy()
# Encode categorical variables
le = LabelEncoder()
X['fuel_type'] = le.fit_transform(X['fuel_type'])
# Store feature names
co2_feature_names = ["Fuel Type", "Year", "Engine Capacity"]
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("CO2 EEA model data shape", X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# Create and train the model
rf_model_co2_eea = RandomForestRegressor(
    n_estimators=200,
    max_depth=None,
    min_samples_split=25,
    min_samples_leaf=5,
    random_state=42,
    n_jobs=-1
)
rf_model_co2_eea.fit(X_train, y_train)

# Calculate and print R-squared for mass model
mass_train_r2 = rf_model_co2_eea.score(X_train, y_train)
mass_test_r2 = rf_model_co2_eea.score(X_test, y_test)
print(f"\nMass Model R-squared - Training: {mass_train_r2:.4f}, Testing: {mass_test_r2:.4f}")

# Get and print feature importance for mass model
co2_eea_importance = rf_model_co2_eea.feature_importances_
print("\nMass Model Feature Importance:")
for name, imp in zip(co2_feature_names, co2_eea_importance):
    print(f"{name}: {imp:.4f}")
print("\n")   

# Finally plot the two models predictons vs test
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

# CO2 model
y_pred = rf_model_co2_eea.predict(X_test)
ax1.hexbin(y_test, y_pred, gridsize=50, cmap='Blues', extent=(0, 350, 0, 350))
ax1.set_xlim(0, 350)
ax1.set_ylim(0, 350)
ax1.set_ylabel("Predicted CO2 g/km")
ax1.set_xlabel("Actual CO2 g/km")
  
# Mass model
# y_pred_mass = rf_model_mass.predict(X_test_mass)
ax2.hexbin(y_test_mass, y_pred_mass, gridsize=50, cmap='Blues', extent=(0, 3000, 0, 3000))
ax2.set_xlim(0, 3000)
ax2.set_ylim(0, 3000)
ax2.set_ylabel("Predicted mass (kg)")
ax2.set_xlabel("Actual mass (kg)")

plt.show()


In [ ]:
# 2. Now pre-process the MOT data - add new cols for CO2 per km, mass, battery size, fuel efficiency, age, and age rounded  
# Loop over MOT data by year
# MOT data from https://www.data.gov.uk/dataset/e3939ef8-30c7-4ca8-9c7c-ad9475cc9b2f/anonymised_mot_test
for year in tqdm(list(range(2023, 2004, -1)) ):
    print(year)

    # Columns to read from the MOT reuslts data
    results_columns = [
        "vehicle_id",
        "test_date",
        "test_class_id",
        "test_result",
        "test_mileage",
        "make",
        "model",
        "fuel_type",
        "cylinder_capacity",
        "first_use_date",
        ]

    # Get MOT 'Results' 
    directory = str(year) + "_Result"
    filename = f"{year}_all_results.csv"
    file_path =  os.path.join(data_path, directory, filename)
    print(f"Input: {filename}")
    df = pd.read_csv(file_path, engine='pyarrow', dtype_backend='pyarrow', usecols=results_columns)
    
    # print number of unique vehicles for each fuel type
    for fuel in ["EL", "HY", "PE", "DI"]:
        n_vehicles = df.loc[df["fuel_type"] == fuel, "vehicle_id"].nunique()
        print(f"In {year}, number of unique {fuel} vehicles: {n_vehicles}")
        
    print("Preprocessing the MOT data")
    
    print("\tCylinder capacity")
    # convert cylinder_capacity to numeric
    df["cylinder_capacity"] = pd.to_numeric(df["cylinder_capacity"], errors='coerce')
    df["cylinder_capacity"] = df["cylinder_capacity"].fillna(0) # catch nans for BEVs
    df["cylinder_capacity_rounded"] = df["cylinder_capacity"].apply(lambda x: round(x, -2)) # used for VCA data matching PHEVs
    # change any EL cylinder_capacity > 0 to HY
    df.loc[(df["cylinder_capacity"] > 0) & (df["fuel_type"]=="EL"), "fuel_type"] = "HY"
    # Now rename other fuel types to be consistent
    
    print("\tFuel Type Labels")
    df.loc[df["fuel_type"] == "Electric", "fuel_type"] = "EL"
    df.loc[df["fuel_type"] == "Hybrid Electric (Clean)", "fuel_type"] = "HY"
    df.loc[df["fuel_type"] == "ED", "fuel_type"] = "HY" # Electric Diesel Hybrids
    # Then only keep the fuels we are interested in (excluding Diesel Hyrbids)
    df = df[df["fuel_type"].isin(["EL", "HY", "PE", "DI"])]
    
    # Convert dates in datetime objects & get age
    print("\tDatetime columns")
    df['first_use_date'] = pd.to_datetime(df['first_use_date'], format="%Y-%m-%d", errors='coerce')
    # Cap first use date to 1980 (25 years before the earliest MOT data)
    df.loc[df['first_use_date'].dt.year < 1980, 'first_use_date'] = pd.to_datetime("1980-01-01", format="%Y-%m-%d", errors='coerce')
    df['test_date'] = pd.to_datetime(df['test_date'], format="%Y-%m-%d", errors='coerce')
    df['first_use_year'] = df['first_use_date'].dt.year
    df['test_year'] = df['test_date'].dt.year
    # Drop rows with missing dates
    df = df[df['first_use_date'].notna()]
    df = df[df['test_date'].notna()]
    # Calc age in years
    df['age_day'] = df['test_date'] - df['first_use_date']
    df['age_year'] = df['age_day'].dt.days / 365.25
    df['age_year_rounded'] = round(df['age_year'])
    
    # Next match the MOT to the vehicle databases
    print("Matching the MOT data to the vehicle databases")
    # Matching is seperate for EL vs DI PE HY 
    # And set up new col for CO2 (mass col is added with the merge with bev spec)
    df["CO2 g/km"] = 0.0

    # Merge the BEV data
    df = df.merge(bev_spec, on=["first_use_year", "make", "model"], how="left")
    
    # Next apply the random forest model to the data
    print("Applying the RFR CO2 model to the data")
    
    # Get mask of EL and HY
    mask_EL = df["fuel_type"] == "EL"  
    mask_HY = df["fuel_type"] == "HY"
    
    # Apply the RFR model for CO2 g/km 
    # first drop any rows with missing values for prediction
    # Drop rows with missing values for the specified columns
    cols = ["fuel_type", "first_use_year", "cylinder_capacity"]
    df = df.dropna(subset=cols)
    # Get the data in the correct format
    X = df.loc[~mask_EL].copy()
    X = X[cols]
    #   Replace firt use years below 2001 with 2001
    X.loc[X["first_use_year"] < 2001, "first_use_year"] = 2001
    X['fuel_type'] = le.transform(X['fuel_type'])
    # rename columns to match training data
    X.columns = ["fuel_type", "Year", "Engine Capacity"]
    y = rf_model_co2.predict(X)
    # Add the predicted CO2 values to the df
    df.loc[~mask_EL, "CO2 g/km"] = y
    
    # Apply the RFR model for mass
    print("Applying the RFR mass model to the data")
    # first drop any rows with missing values for prediction
    cols = ["fuel_type", "first_use_year", "cylinder_capacity", "CO2 g/km"]
    df = df.dropna(subset=cols)
    # Get the data in the correct format
    X = df.loc[~mask_EL].copy()
    X = X[["fuel_type", "first_use_year", "cylinder_capacity", "CO2 g/km"]]
    #   Replace firt use years below 2010 with 2010
    X.loc[X["first_use_year"] < 2010, "first_use_year"] = 2010
    X['fuel_type'] = le.transform(X['fuel_type'])
    # rename columns to match training data
    X.columns = ["fuel_type", "Year", "Engine Capacity", "CO2 g/km"]
    y = rf_model_mass.predict(X)
    # Add the predicted mass values to the df
    df.loc[~mask_EL, "mass (kg)"] = y
    
    # Finally merge the vca data to remove PHEVs from the data
    # This requiires time intesnive processing of the MOT data make and model cols to match the VCA data
    print("Prep MOT data for VCA merge for PHEV identification")
    # Models are given first, then followed by trim descriptions
    # Split the strings after the main model name - eg "GOLF GTI" -> "GOLF"
    # Split the df up to make this much faster
    df_HY = df.loc[mask_HY].copy()
    df = df.loc[~mask_HY].copy() 
    # Exhaustive list of model names to remove detail not included in vca data - not necessary for all PHEVs but keep for now
    df_HY["model"] = df_HY["model"].str.split("GT", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split("CRDI", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split("CDI", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split("TDI", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split("TSI", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split("TFSI", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split("TDCI", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split("R-DYN", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split("HEV", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split("MHEV", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split("PHEV", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split("HYBRID", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split(" PLUS", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split("+", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split("RANGE EXTENDER", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split("EXCEL", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split("DESIGN", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split("TOURING SPORTS", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split("ICON", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split("LIME EDITION ", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split("ORANGE EDITION ", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split("BLACK EDITION ", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split("LAUNCH EDITION ", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split("GR SPORT", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split("TREK", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split("ESTATE", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split("INVINCIBLE", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split("PRO AUTO", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split("CITY", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split("S-A", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split("PREMIUM", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split("LUXURY ", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split(" SE ", n=1).str[0] # include spaces to avoid removing correct model names
    df_HY["model"] = df_HY["model"].str.split("FIRST EDITION", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split("ULTIMATE", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split("ST-LINE", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split("TITANIUM", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split("VIGNALE", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split("ACTIVE", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split("TREND", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split("MATCH", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split("FLYING", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split("SPEED", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split("CARBON", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split("VORSPRUNG", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split("TECHNIK", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split("S LINE", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split("S-LN", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split("S LN", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split("SLN", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split("CABRIOLET", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split("CITYCARVER", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split("AUTO", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split("MOMENTUM", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split("INSCRIPTION", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split("R-DESIGN", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split("AWD", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split("4WD", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split("M SPORT", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split("XDRIVE", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split("AIRCORSS", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split("CBACK", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split("CROSSBACK", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split("C-BACK", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split("CROSSTAR", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split(" EX ", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split(" SR ", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split(",", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split("R-DYNAMIC", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split("S P300E", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split("HSE P300E", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split("EXECUTIVE", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split(" 110 ", n=1).str[0] # MERC numbers:
    df_HY["model"] = df_HY["model"].str.split(" 114 ", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split(" 116 ", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split(" 160 ", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split(" 180 ", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split(" 200 ", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split(" 250 ", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split(" 300 ", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split(" 450 ", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split(" VISTA ", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split(" R-LINE  ", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.split(" SEL", n=1).str[0]
    df_HY["model"] = df_HY["model"].str.replace("90 R-", "90")
    df_HY["model"] = df_HY["model"].str.replace("90 R", "90")
    df_HY["model"] = df_HY["model"].str.replace("60 R-", "90")
    df_HY["model"] = df_HY["model"].str.replace("60 R", "90")
    df_HY["model"] = df_HY["model"].str.replace("40 R-", "90")
    df_HY["model"] = df_HY["model"].str.replace("40 R", "90")
    # Specific to Land Rover - inconsistent naming in the data for a large number of vehicles
    df_HY.loc[df_HY["model"].str.contains("DISCOVERY SPRT", na=False), "model"] = "DISCOVERY SPORT"
    df_HY.loc[df_HY["model"].str.contains("DISCOVRY SPRT", na=False), "model"] = "DISCOVERY SPORT"
    df_HY.loc[df_HY["model"].str.contains("R ROVER EVOQUE", na=False), "model"] = "RANGE ROVER EVOQUE"
    df_HY.loc[df_HY["model"].str.contains("R ROVER SPORT", na=False), "model"] = "RANGE ROVER SPORT"
    df_HY.loc[df_HY["model"].str.contains("R ROVER SPRT", na=False), "model"] = "RANGE ROVER SPORT"
    # Likewise for MERCEDES - inconsistent naming for A-Class HEVs (20,000 HEVs)
    df_HY.loc[(df_HY["fuel_type"] == "car_hev") & (df_HY["cylinder_capacity"]== 1332), "model"] = "A-CLASS"
    # Also change all A to A-CLASS, B to B-CLASS, etc
    df_HY.loc[(df_HY["make"] == "MERCEDES-BENZ") & (df_HY["model"] == "A"), "model"] = "A-CLASS"
    df_HY.loc[(df_HY["make"] == "MERCEDES-BENZ") & (df_HY["model"] == "B"), "model"] = "B-CLASS"
    df_HY.loc[(df_HY["make"] == "MERCEDES-BENZ") & (df_HY["model"] == "C"), "model"] = "C-CLASS"
    df_HY.loc[(df_HY["make"] == "MERCEDES-BENZ") & (df_HY["model"] == "E"), "model"] = "E-CLASS"
    df_HY.loc[(df_HY["make"] == "MERCEDES-BENZ") & (df_HY["model"] == "S"), "model"] = "S-CLASS"
    # Change MINI COOPER to MINI MINI
    df_HY.loc[(df_HY["make"] == "MINI") & (df_HY["model"] == "COOPER"), "model"] = "MINI"
    # For BMW, split at M and take the first part
    df_HY.loc[(df_HY["make"] == "BMW"), "model"] = df_HY.loc[(df_HY["make"] == "BMW"), "model"].str.split("M", n=1).str[0]
    # Not for land rover or KIA, split at sport and take the first part 
    # avoiding: "RANGE ROVER SPORT" -> "RANGE ROVER", KIA "SPORTAGE" -> "SPORTAGE"
    df_HY.loc[(df_HY["make"] != "LAND ROVER") & (df_HY["make"] != "KIA"), "model"] = df_HY.loc[(df_HY["make"] != "LAND ROVER") & (df_HY["make"] != "KIA"), "model"].str.split("SPORT", n=1).str[0]
    # Finally remove any trailing or leading whitespace
    df_HY["model"] = df_HY["model"].str.strip()
    df_HY["make"] = df_HY["make"].str.strip()
    # Add the data back together
    df = pd.concat([df, df_HY], ignore_index=True)
    
    # merge the data with the PHEV lookup
    df = df.merge(vca_phev_lookup, 
             how='left', 
             left_on=['make', 'model', 'first_use_year', 'cylinder_capacity_rounded', 'fuel_type'], 
             right_on=['Manufacturer', 'Model', 'Year', 'Engine Capacity', 'fuel_type']
             )
    
    # print number of unique vehicles for each fuel type
    for fuel in df["fuel_type"].unique():
        n_vehicles = df.loc[df["fuel_type"] == fuel, "vehicle_id"].nunique()
        print(f"After pre-processing {year}, number of unique {fuel} vehicles: {n_vehicles}")
        if fuel == "HY":
            n_phevs = df.loc[(df["fuel_type"] == fuel) & (df["ev_type"] =="phev"), "vehicle_id"].nunique()
            print(f"Number of PHEVs: {n_phevs} (2015-2020 Models)")
    
    # Convert to parquet - speeds up later steps where only subsets (cols) of the data are needed
    parquet_filename = f"{year}_all_results.parquet"
    parquet_file_path = os.path.join(data_path, directory, parquet_filename)
    df.to_parquet(parquet_file_path, engine='pyarrow', compression='snappy')
    print(f"Output: {parquet_filename}")

In [ ]:
# 3. Create Vehicle Sample using vehicle_id
# The cohort is defined as vehicles that are 3 years old in the given year
# I.e., the first tests for these vehicles when they enter the MOT data

all_ids = {
    "lgv": [], # note not used in the paper 
    "car_diesel": [],
    "car_petrol": [],
    "car_hev": [],
    "car_phev": [],
    "car_bev": [],
    "mcycle" : [], # note not used in the paper 
}

for year in tqdm(list(range(2023, 2004, -1)) ):
    print(year)
    directory = str(year) + "_Result"
    filename = f"{year}_all_results.parquet"
    file_path =  os.path.join(data_path, directory, filename)
    print(f"Input: {filename}")
    # Only read in minimum number of cols for faster processing of IDs
    columns = ['vehicle_id', 'age_year_rounded', 'test_class_id', 'make', 'model', 'fuel_type', 'ev_type', 'battery capacity (kWh)', 'cylinder_capacity']
    df = pd.read_parquet(file_path, engine='pyarrow', columns=columns)
    
    # Filter for 3 year old vehicles - this is the cohort for the test year
    if year in [2005, 2006]:
        df = df[(df['age_year_rounded'] >= 3) & (df['age_year_rounded'] <= 35)]
        sample_size = 1_000_000 # larger sample size as ages 3-30 are included
        # Note that this sample size is approx the same as the total number of vehicles in the MOT data for 2005 thus include 2006 as well
    else:
        df = df[df['age_year_rounded'] == 3]  
        sample_size = 200_000 # only age 3 vehicles are included # only used for scrappage and mileage figures = not in training 

    # Get LGV IDs
    print("\tLGVs")

    # Filter df for LGVs predefined makes and models
    # Remommneded by DfT since some LGVs are in class 4 and rest are in class 7
    df_sample = df[
        (
            
        # LGVs in class 4
        (df['test_class_id'] == 4) & 
        (  ((df['make'].str.contains('FORD')) &
            (df['model'].str.contains("TRANSIT"))) |

            ((df['make'].str.contains('MERCEDES')) &
            (df['model'].str.contains("SPRINTER"))) |

            ((df['make'].str.contains('MERCEDES')) &
            (df['model'].str.contains("VITO"))) |

            ((df['make'].str.contains('VAUXHALL')) &
            (df['model'].str.contains("VIVARO"))) |

            ((df['make'].str.contains('VAUXHALL')) &
            (df['model'].str.contains("COMBO"))) |

            ((df['make'].str.contains('VAUXHALL')) &
            (df['model'].str.contains("MOVANO"))) |

            ((df['make'].str.contains('VOLKS')) &
            (df['model'].str.contains("TRANSPORTER"))) |

            ((df['make'].str.contains('VOLKS')) &
            (df['model'].str.contains("CADDY"))) |

            ((df['make'].str.contains('CITROEN')) &
            (df['model'].str.contains("RELAY"))) |

            ((df['make'].str.contains('CITROEN')) &
            (df['model'].str.contains("DISPATCH"))) |

            ((df['make'].str.contains('CITROEN')) &
            (df['model'].str.contains("RELAY"))) |

            ((df['make'].str.contains('CITROEN')) &
            (df['model'].str.contains("Berlingo"))) |

            ((df['make'].str.contains('PEUGEOT')) &
            (df['model'].str.contains("PARTNER"))) |

            ((df['make'].str.contains('PEUGEOT')) &
            (df['model'].str.contains("BOXER"))) |

            ((df['make'].str.contains('RENAULT')) &
            (df['model'].str.contains("TRAFIC"))) |

            ((df['make'].str.contains('RENAULT')) &
            (df['model'].str.contains("KANGOO"))) |

            ((df['make'].str.contains('FIAT')) &
            (df['model'].str.contains("DUCATO")))
        )
        
        # LGVs in class 7 
        ) |
        (df['test_class_id'] == 7) 
        ]
    
    print(f"\t{year} Total LGVs: {df_sample['vehicle_id'].nunique()}")
    
    all_ids["lgv"].extend(df_sample['vehicle_id'].unique())

    # Now get ID sample from filtered DF
    sample_lgv = df_sample['vehicle_id'].unique()
    sample_lgv = pd.DataFrame(sample_lgv, columns=['vehicle_id'])
    
    # only keep sample_size LGVs
    if len(sample_lgv) > sample_size:
        sample_lgv = sample_lgv.sample(n=sample_size, random_state=42)

    directory = "Sample_Data" 
    filename = f"sample_IDs_lgv_{year}.parquet"
    file_path =  os.path.join(data_path, directory, filename)
    sample_lgv.to_parquet(file_path, engine='pyarrow', compression='snappy') 
    print(f"\tLGV Sample: {len(sample_lgv)}")

    # Remove LGVs from vehicle class 4
    # Now only cars (mostly) remain in class 4 - see MOT docs
    df = df [~df['vehicle_id'].isin(sample_lgv['vehicle_id'])]  

    # Next Cars
    # Diesel Cars first 
    print("\tDiesel Cars")
    df_sample = df[(df['test_class_id'] == 4) & (df['fuel_type'] == 'DI')]
    print(f"\t{year} Total DI: {df_sample['vehicle_id'].nunique()}")
    all_ids["car_diesel"].extend(df_sample['vehicle_id'].unique())

    # Now get ID sample from filtered DF
    sample_di_car = df_sample['vehicle_id'].unique()
    sample_di_car = pd.DataFrame(sample_di_car, columns=['vehicle_id'])
    
    # only keep sample_size DE
    if len(sample_di_car) > sample_size:
        sample_di_car = sample_di_car.sample(n=sample_size, random_state=42)
            
    directory = "Sample_Data"
    filename = f"sample_IDs_car_diesel_{year}.parquet"
    file_path =  os.path.join(data_path, directory, filename)
    sample_di_car.to_parquet(file_path, engine='pyarrow', compression='snappy') 
    print(f"\tDiesel Car Sample: {len(sample_di_car)}")

    # Next Petrol Cars
    print("\tPetrol Cars")
    df_sample = df[(df['test_class_id'] == 4) &(df['fuel_type'] == 'PE') ]
    print(f"\t{year} Total PE: {df_sample['vehicle_id'].nunique()}")
    all_ids["car_petrol"].extend(df_sample['vehicle_id'].unique())

    # Now get ID sample from filtered DF
    sample_pe_car = df_sample['vehicle_id'].unique()
    sample_pe_car = pd.DataFrame(sample_pe_car, columns=['vehicle_id'])
    
    # only keep sample_size DE
    if len(sample_pe_car) > sample_size:
        sample_pe_car = sample_pe_car.sample(n=sample_size, random_state=42)
        
    directory = "Sample_Data"
    filename = f"sample_IDs_car_petrol_{year}.parquet"
    file_path =  os.path.join(data_path, directory, filename)
    sample_pe_car.to_parquet(file_path, engine='pyarrow', compression='snappy') 
    print(f"\tPetrol Car Sample: {len(sample_pe_car)}")

    # Next HEVs -  Sample is the entire HEV fleet - excluding PHEVs
    print("\tHEV Cars")
    df_phevs_sample = df[(df['test_class_id'] == 4) & (df['fuel_type'] == 'HY') & (df['ev_type'] == 'phev')]
    df_sample = df[(df['test_class_id'] == 4) & (df['fuel_type'] == 'HY') & (df['ev_type'] != 'phev')]
    print(f"\t{year} Total HY: {df_sample['vehicle_id'].nunique()}")
    print(f"\t{year} Total PHEVs exlcuded: {df_phevs_sample['vehicle_id'].nunique()}")
    all_ids["car_hev"].extend(df_sample['vehicle_id'].unique())
    all_ids['car_phev'].extend(df[(df['test_class_id'] == 4) & (df['fuel_type'] == 'HY') & (df['ev_type'] == 'phev')]['vehicle_id'].unique())
    
    # Now get ID sample from filtered DF - sample is all HEVS
    sample_hy_car = df_sample[['vehicle_id']]
    sample_hy_car = sample_hy_car.drop_duplicates()
    directory = "Sample_Data"
    filename = f"sample_IDs_car_hev_{year}.parquet"
    file_path =  os.path.join(data_path, directory, filename)
    sample_hy_car.to_parquet(file_path, engine='pyarrow', compression='snappy') 
    print(f"\tHEV Car Sample: {len(sample_hy_car)}")

    # BEV Cars -  Sample is the entire BEV fleet
    print("\tBEV Cars")
    df_sample = df[(df['test_class_id'] == 4)&(df['fuel_type'] == 'EL')]
    print(f"\t{year} Total EL: {df_sample['vehicle_id'].nunique()}")
    
    # Only keep vehicles if they have Battery Size data
    df_sample = df_sample[~df_sample['battery capacity (kWh)'].isna()]
    
    # Now get sample from filtered DF
    sample_el_car = df_sample[['vehicle_id']]
    sample_el_car = sample_el_car.drop_duplicates()
    directory = "Sample_Data"
    filename = f"sample_IDs_car_bev_{year}.parquet"
    file_path =  os.path.join(data_path, directory, filename)
    sample_el_car.to_parquet(file_path, engine='pyarrow', compression='snappy') 
    print(f"\tBEV Car Sample: {len(sample_el_car)}")
    all_ids["car_bev"].extend(df[(df['test_class_id'] == 4) & (df['fuel_type'] == 'EL')]['vehicle_id'].unique())
    
    # for refrence output bevs that are not in the sample
    directory = "Sample_Data"
    fname = f"BEV_not_in_sample_{year}.csv"
    fpath = os.path.join(data_path, directory, fname)
    df[df["vehicle_id"].isin(all_ids["car_bev"]) == False].to_csv(fpath, index=False)
    
    # Motorcycles 
    print("\tMcycles")

    df_sample = df[(df['test_class_id'] == 1) | (df['test_class_id'] == 2)]
    print(f"\t{year} Total MCycle: {df_sample['vehicle_id'].nunique()}")
    # Now get ID sample from filtered DF
    sample_mcycle = df_sample['vehicle_id'].unique()  
    sample_mcycle = pd.DataFrame(sample_mcycle, columns=['vehicle_id'])
    
    # only keep sample_size Motorcycles
    if len(sample_mcycle) > sample_size:
        sample_mcycle = sample_mcycle.sample(n=sample_size, random_state=42)
    
    directory = "Sample_Data"
    filename = f"sample_IDs_mcycle_{year}.parquet"
    file_path =  os.path.join(data_path, directory, filename)
    sample_mcycle.to_parquet(file_path, engine='pyarrow', compression='snappy') 
    print(f"\tMotorcycle Sample: {len(sample_mcycle)}")
    all_ids["mcycle"].extend(df_sample['vehicle_id'].unique())
    

    # Clear memory - reduces RAM for intial input of next year's dfs
    del [
        df, sample_lgv, sample_di_car, sample_pe_car, 
        sample_hy_car, sample_el_car, sample_mcycle]
    gc.collect()

for fuel in all_ids.keys():
    # Make sure there are no duplicates
    all_ids[fuel] = list(set(all_ids[fuel]))
    print(f"{fuel}: {len(all_ids[fuel])}")

# Open MOT data for each year and save the data for the sample IDs
# This reduces the size of the data to be processed in the next steps
# And acts as a first set of filtering as we want only vehicles with their first test at 3 years old

# IDs for LGVs, Diesel Cars, Petrol Cars, HEV Cars, BEV Cars, Motorcycles
# List holds ID df for each year
# Then concatenate dfs and drop duplicates to get unique IDs
lgvs_ids = []
car_diesel_ids = []
car_petrol_ids = []
car_hev_ids = []
car_bev_ids = []
mcycle_ids = []

# Change dir to "Sample_Data" to get sample IDs - makes below code more readable
print("Getting Sample IDs")
os.chdir("Sample_Data")
for year in range(2005, 2024):
    print(year)
    lgvs_ids.append(pd.read_parquet(f"sample_IDs_lgv_{year}.parquet", engine='pyarrow'))
    car_diesel_ids.append(pd.read_parquet(f"sample_IDs_car_diesel_{year}.parquet", engine='pyarrow'))
    car_petrol_ids.append(pd.read_parquet(f"sample_IDs_car_petrol_{year}.parquet", engine='pyarrow'))
    car_hev_ids.append(pd.read_parquet(f"sample_IDs_car_hev_{year}.parquet", engine='pyarrow'))
    car_bev_ids.append(pd.read_parquet(f"sample_IDs_car_bev_{year}.parquet", engine='pyarrow'))
    mcycle_ids.append(pd.read_parquet(f"sample_IDs_mcycle_{year}.parquet", engine='pyarrow'))

lgvs_ids = pd.concat(lgvs_ids).drop_duplicates()['vehicle_id'].to_list()
car_diesel_ids = pd.concat(car_diesel_ids).drop_duplicates()['vehicle_id'].to_list()
car_petrol_ids = pd.concat(car_petrol_ids).drop_duplicates()['vehicle_id'].to_list()
car_hev_ids = pd.concat(car_hev_ids).drop_duplicates()['vehicle_id'].to_list()
car_bev_ids = pd.concat(car_bev_ids).drop_duplicates()['vehicle_id'].to_list()
mcycle_ids = pd.concat(mcycle_ids).drop_duplicates()['vehicle_id'].to_list()

# Change dir back to script path
os.chdir(data_path)

# Print sample counts
print("ID sample count")
print("LGV:", len(lgvs_ids))
print("Diesel Car:", len(car_diesel_ids))
print("Petrol Car:", len(car_petrol_ids))
print("HEV Car", len(car_hev_ids))
print("BEV Car", len(car_bev_ids))
print("Mcycle", len(mcycle_ids))

# Now get MOT data using sample IDs
for year in tqdm(MOT_data_years):
    print(year)

    directory = str(year) + "_Result"
    filename = f"{year}_all_results.parquet"
    file_path = os.path.join(data_path, directory, filename)
    print(f"\tInput: {filename}")
    df = pd.read_parquet(file_path, engine='pyarrow')

    all_ids = [lgvs_ids, car_diesel_ids, car_petrol_ids, car_hev_ids, car_bev_ids, mcycle_ids]
    veh_names = ["lgv", "car_diesel", "car_petrol", "car_hev", "car_bev", "mcycle"]
    for veh_ids, veh_name in zip(all_ids, veh_names):
        
        # filter MOT data for IDs
        df_filter = df[df['vehicle_id'].isin(veh_ids)]
        print(f"\t\t{veh_name}: {len(df_filter)}")
        directory = "Sample_Data"
        filename = f"sample_{veh_name}_MOT_{year}.parquet"
        file_path = os.path.join(data_path, directory, filename)
        print(f"\t\tOutput: {filename}")
        df_filter.to_parquet(file_path, engine='pyarrow', compression='snappy')
        
del [df, df_filter, lgvs_ids, car_diesel_ids, car_petrol_ids, car_hev_ids, car_bev_ids, mcycle_ids]
gc.collect()

In [ ]:
# 4. Aggregate data for Mileage & Scrappage Analysis + Transfromer Model

fuel_lookup = {
    "car_hev" : "HY",
    "car_bev" : "EL",
    "car_diesel" : "DI",
    "car_petrol" : "PE",
}

veh_names = ["car_diesel", "car_petrol", "car_hev", "car_bev"]
for veh_name in tqdm(veh_names):
    print(veh_name)
    
    # Read in Data
    years = list(range(2005, 2024, 1))
    df = []
    for year in years:
        print(f"Reading in MOT Data from: {year}")
        directory = "Sample_Data"
        filename = f"sample_{veh_name}_MOT_{year}.parquet"
        file_path = os.path.join(data_path, directory, filename)
        df_year = pd.read_parquet(file_path, engine='pyarrow')
        # drop cols that are not needed from now on
        drop_cols = [
            'Fuel Type', 'Manufacturer', 'Model', 'Engine Capacity', 'Year',
            'cylinder_capacity_rounded', 'test_class_id',
        ]
        df_year.drop(columns=drop_cols, inplace=True)
        
        # Make sure that only correct fuel type is in the data
        df_year = df_year[df_year['fuel_type'] == fuel_lookup[veh_name]]
        
        df.append(df_year)
        
    print(f"Concatenating {veh_name} Samples from MOT Years")    
    df = pd.concat(df, ignore_index=True)
    df = df.reset_index(drop=True)
    
    del df_year
    gc.collect()
    
    print(df.shape)
    print("Sample before processign steps:", df['vehicle_id'].nunique())
    
    n = df['vehicle_id'].nunique()
    # Remove any vehicles that do not have 'cylinder_capacity' (excl. BEVs)
    if veh_name != "car_bev":
        ids = df.loc[(df['cylinder_capacity'].isna()) | (df['cylinder_capacity'] == 0) | (df['cylinder_capacity'] < 0), 'vehicle_id'].unique()
        df = df[~df['vehicle_id'].isin(ids)]
        new_n = df['vehicle_id'].nunique()
        print(f"Removed {n - new_n} vehicles with missing cylinder_capacity")
        n = new_n

    # Ensure relevant columns are numeric
    df['age_year'] = pd.to_numeric(df['age_year'], errors='coerce')
    df['test_mileage'] = pd.to_numeric(df['test_mileage'], errors='coerce')

    # Drop vehicle_ids with missing test_mileage or age - will create errors in later steps
    n = df['vehicle_id'].nunique()
    ids = df.loc[df['test_mileage'].isna(), 'vehicle_id'].unique()
    df = df[~df['vehicle_id'].isin(ids)]
    new_n = df['vehicle_id'].nunique()
    print(f"Removed {n - new_n} vehicles with missing test_mileage")
    n = new_n
    ids = df.loc[df['age_year'].isna(), 'vehicle_id'].unique()
    df = df[~df['vehicle_id'].isin(ids)]
    new_n = df['vehicle_id'].nunique()
    print(f"Removed {n - new_n} vehicles with missing age_year")
    n = new_n

    # Sort by vehicle ID and test date
    print("Sorting by vehicle ID and test date")
    df = df.sort_values(by=['vehicle_id', 'test_date'])

    print("Removing follow-up tests")
    # Next remove tests if they are not a pass, fail or PRS
    # ABA and ABR result in another test - therefore just exlcude these
    df = df[df['test_result'].isin(['P', 'F', 'PRS'])] 
    # Now get mileage per year, time between tests, etc
    # Time between tests - in years
    df[f"previous_age_year"] = df.groupby("vehicle_id")["age_year"].shift(1)
    # For the first test for each vehicle_id set the previous_age_year to current age_year -1
    df.loc[df["vehicle_id"] != df["vehicle_id"].shift(1), "previous_age_year"] = df["age_year"] - 1
    df["time_between_tests"] = df["age_year"] - df["previous_age_year"]
    # Now remove any tests that are too close together - likely due to a F followed by a P a few days later
    df = df[(df['time_between_tests'] > 0.2) ] # remove tests that are less than 0.2 years (~ 2 months) apart
    # Drop the new columns as these are recalculated later
    df.drop(columns=['previous_age_year', 'time_between_tests'], inplace=True)
    
    # Remove any tests before 3 years old
    df = df[df['age_year_rounded'] >= 3] # keep tests over 3 years old - assume first test is at 3 years old
    
    # Use shifts to calculate changes between tests
    print("Calculating changes between tests")
    # Shift the data to find the changes between tests
    df[f"previous_test_mileage"] = df.groupby("vehicle_id")["test_mileage"].shift(1)
    df[f"previous_test_date"] = df.groupby("vehicle_id")["test_date"].shift(1)
    df[f"previous_age_year"] = df.groupby("vehicle_id")["age_year"].shift(1)
    df[f"previous_age_year_rounded"] = df.groupby("vehicle_id")["age_year_rounded"].shift(1) # probably not useful - need to round again later
    # Check if the row is the first or last test in the data for each vehicle id using shift
    df['last_test'] = df['vehicle_id'] != df['vehicle_id'].shift(-1)
    df['first_test'] = df['vehicle_id'] != df['vehicle_id'].shift(1)
    
    # Now drop the first and last vehicle_id as the shift function will not work for these
    df = df[df["vehicle_id"] != df["vehicle_id"].iloc[0]]
    df = df[df["vehicle_id"] != df["vehicle_id"].iloc[-1]]
    
    # Now get mileage per year, time between tests, etc
    # Time between tests - in years
    df["time_between_tests"] = df["age_year"] - df["previous_age_year"]
    # Mileage between test
    df["mileage_between_tests"] = df["test_mileage"] - df["previous_test_mileage"]
    # Change mileage_between_tests to float32 - needed for later calculation
    df['mileage_between_tests'] = df['mileage_between_tests'].astype('float32') # 
    # Mileage per year
    df["mileage_per_year"] = df["mileage_between_tests"] / df["time_between_tests"]
    
    # Now perfrom some checks on these new columns
    # Remove vehicles with mileage_between_tests < 0, i.e. input error for a test
    n = df["vehicle_id"].nunique()
    ids = df.loc[(df["mileage_between_tests"] < 0), "vehicle_id"].unique()
    df = df[~df["vehicle_id"].isin(ids)]
    new_n = df["vehicle_id"].nunique()
    print(f"Removed {n - new_n} vehicles with mileage_between_tests < 0")
    n = new_n
    # Remove vehicles with mileage_per_year < 0, i.e. input error for a test - from time_between_tests being 0
    ids = df.loc[(df["mileage_per_year"] < 0), "vehicle_id"].unique()
    df = df[~df["vehicle_id"].isin(ids)]
    new_n = df["vehicle_id"].nunique()
    print(f"Removed {n - new_n} vehicles with mileage_per_year < 0")
    n = new_n
    # Repeat  for time_between_tests - exclude vehicles with time_between_tests < 0
    ids = df.loc[(df["time_between_tests"] < 0), "vehicle_id"].unique()
    df = df[~df["vehicle_id"].isin(ids)]
    new_n = df["vehicle_id"].nunique()
    print(f"Removed {n - new_n} vehicles with time_between_tests < 0")
    
    # Now correct the rows for the vehicle's first_test
    df.loc[df["first_test"] == True, "mileage_per_year"] = df["test_mileage"] / df["age_year"]
    df.loc[df["first_test"] == True, "time_between_tests"] = df["age_year"]
    df.loc[df["first_test"] == True, "mileage_between_tests"] = df["test_mileage"]  
    # And add some corrections for these tests to make the data as if it has been 1 year since a previous test
    # Edit the time between tests to 1.0 for first_test == True and age_year_rounded == 3 
    df.loc[(df["age_year_rounded"] == 3) & (df["first_test"]==True), "time_between_tests"] = 1.0 
    df.loc[(df["age_year_rounded"] == 3) & (df["first_test"]==True), "mileage_between_tests"] *= 0.333 # divide by 3
    df.loc[(df["test_year"] <= 2006) & (df["first_test"]==True), "time_between_tests"] = 1.0 # set to 1.0 for before 2006 
    
    # Check for errors from the shifts
    print("Checking for errors in the df")
    n = df["vehicle_id"].nunique()
    # Check for infinite values
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    if np.isinf(df[numeric_cols]).values.any():
        print("df contains infinite values. Replacing them with NaN.")
        df[numeric_cols] = df[numeric_cols].replace([np.inf, -np.inf], np.nan)
    new_n = df["vehicle_id"].nunique()
    print(f"Removed {n - new_n} vehicles with infinite values")
    n = new_n
           
    # Remove any ids with NaN values in these columns
    n = df["vehicle_id"].nunique()
    ids = df.loc[df['mileage_per_year'].isna(), 'vehicle_id'].unique()
    df = df[~df['vehicle_id'].isin(ids)]
    new_n = df["vehicle_id"].nunique()
    print(f"Removed {n - new_n} vehicles with mileage per year < 100")
    n = new_n
    ids = df.loc[df['time_between_tests'].isna(), 'vehicle_id'].unique()
    df = df[~df['vehicle_id'].isin(ids)]
    new_n = df["vehicle_id"].nunique()
    print(f"Removed {n - new_n} vehicles with time_between_tests np.nan")
    n = new_n
    ids = df.loc[df['mileage_between_tests'].isna(), 'vehicle_id'].unique()
    df = df[~df['vehicle_id'].isin(ids)]
    new_n = df["vehicle_id"].nunique()
    print(f"Removed {n - new_n} vehicles with mileage_between_tests np.nan")
    n = new_n
    
    # Next round mileage cols to ints
    df['test_mileage'] = df['test_mileage'].round(0).astype('int32')
    df['mileage_between_tests'] = df['mileage_between_tests'].round(0).astype('int32')
    df['mileage_per_year'] = df['mileage_per_year'].round(0).astype('int32')
    
    # Next remove any vehicle that has less than 100 miles per year - likely input error missing a digit
    n = df["vehicle_id"].nunique()
    ids = df.loc[df['mileage_per_year'] < 100, 'vehicle_id'].unique()
    
    df = df[~df['vehicle_id'].isin(ids)]
    new_n = df["vehicle_id"].nunique()
    print(f"Removed {n - new_n} vehicles with mileage per year < 100")
    n = new_n
    
    # Now add estimated data for the first and second years based on the first test
    # Year 1 is 1/3 of the third test and year 2 is 2/3 of the third test
    # New rows for years 1 and 2
    print("Adding Rows for Years 0,  1 and 2")
    
    # keep only vehicles that still have their first test
    ids = df.loc[df["first_test"]==True, "vehicle_id"].unique()
    df = df[df["vehicle_id"].isin(ids)]
    new_n = df["vehicle_id"].nunique()
    print(f"Removed {n - new_n} vehicles with first test")
    n = new_n
    
    df_year_3 = df[(df["age_year_rounded"] == 3) & (df["first_test"] == True)]
    
    # Create a new dfframe with the new rows for years 1 and 2
    # Data used for transformer model and CO2 calculations only
    # Year 2
    df_year_2 = df_year_3.copy()
    df_year_2["age_year_rounded"] = 2
    df_year_2["mileage_per_year"] = df_year_2["mileage_per_year"]
    df_year_2["test_mileage"] = 2 * df_year_2["mileage_per_year"]
    df_year_2["test_date"] = df_year_2["test_date"] - dt.timedelta(days=365)
    df_year_2["test_year"] = df_year_2["test_year"] - 1
    df_year_2["time_between_tests"] = 1
    df_year_2["age_year"] = df_year_2["age_year"] - 1
    df_year_2["age_year_rounded"] = 2
    df_year_2["mileage_between_tests"] = df_year_2["mileage_per_year"]
    df_year_2["last_test"] = False
    df_year_2["first_test"] = False
    # Year 1
    df_year_1 = df_year_3.copy()
    df_year_1["age_year_rounded"] = 1
    df_year_1["mileage_per_year"] = df_year_1["mileage_per_year"]
    df_year_1["test_mileage"] = df_year_1["mileage_per_year"]
    df_year_1["test_date"] = df_year_1["test_date"] - dt.timedelta(days=2*365)
    df_year_1["test_year"] = df_year_1["test_year"] - 2
    df_year_1["time_between_tests"] = 1
    df_year_1["age_year"] = df_year_1["age_year"] - 2
    df_year_1["age_year_rounded"] = 1
    df_year_1["mileage_between_tests"] = df_year_1["mileage_per_year"]
    df_year_1["last_test"] = False
    df_year_1["first_test"] = False
    # Year 0 - ie the point of manufacture - used for LCA emissions for manufacture
    df_year_0 = df_year_3.copy()
    df_year_0["age_year_rounded"] = 0
    df_year_0["mileage_per_year"] = 0
    df_year_0["test_mileage"] = 0
    df_year_0["test_date"] = df_year_0["first_use_date"]
    df_year_0["test_year"] = df_year_0["first_use_year"]
    df_year_0["time_between_tests"] = 0
    df_year_0["age_year"] = 0
    df_year_0["age_year_rounded"] = 0
    df_year_0["mileage_between_tests"] = 0
    df_year_0["last_test"] = False
    df_year_0["first_test"] = False
    # Now add these new rows to the df
    df = pd.concat([df, df_year_2, df_year_1, df_year_0], ignore_index=True)
    df = df.sort_values(by=['vehicle_id', 'test_date'])
    
    # drop cols that are not used further
    drop_cols = ['previous_test_mileage', 'previous_test_date', 'previous_age_year', 'previous_age_year_rounded', 'test_result']
    df.drop(columns=drop_cols, inplace=True)
    
    # Finished processing data     
    print(f"Memory usage for dataset- {df.memory_usage().sum() / 1e6:.2f} MB")
    print(df.info())
    print(df.describe().to_string())
    directory = "Sample_Data" 
    filename = f"sample_{veh_name}_all_years.parquet"
    file_path = os.path.join(data_path, directory, filename)
    df.to_parquet(file_path, engine='pyarrow', compression='snappy')
    
    print("Sample after processing steps:", df['vehicle_id'].nunique())
    
    del df
    gc.collect()
    
    

In [ ]:
# 5. Prepare Data for Transformer model:

fuel_lookup = {
    "car_hev" : "HY",
    "car_bev" : "EL",
    "car_diesel" : "DI",
    "car_petrol" : "PE",
}

print("Preparing training data...")
for veh_name in ["car_diesel", "car_petrol"]:
    
    # Only keep the columns we need for training
    categorical_cols = ['fuel_type', 'last_test']
    numerical_cols = ['mileage_per_year', 'test_mileage', 'age_year', 'time_between_tests']
    training_cols = ['vehicle_id', 'test_year', 'first_test', 'first_use_year', 'age_year_rounded'] + categorical_cols + numerical_cols
    
    print(f"\tReading in {veh_name} data...")
    directory = "Sample_Data" 
    filename = f"sample_{veh_name}_all_years.parquet"
    file_path = os.path.join(data_path, directory, filename)
    data = pd.read_parquet(file_path, engine='pyarrow', columns=training_cols)
    
    # Make sure that only correct fuel type is in the data
    ids = data.loc[data['fuel_type'] != fuel_lookup[veh_name], 'vehicle_id'].unique()
    data = data[~data['vehicle_id'].isin(ids)]
    
    
    # Add a few filtering steps to keep data suitable for training
    print(f"\tFiltering {veh_name} Data for Training. Initial Sample: {data['vehicle_id'].nunique()}")
    n = data["vehicle_id"].nunique()
    
    ids = data.loc[data.isnull().any(axis=1), 'vehicle_id'].unique()
    data = data[~data['vehicle_id'].isin(ids)]
    new_n = data["vehicle_id"].nunique()
    print(f"\tRemoved {n - new_n} vehicles with NaN values")
    n = new_n
    
    # Next drop any vehicle that has over 2 Years between tests - outliers
    ids = data.loc[data['time_between_tests'] > 2, 'vehicle_id'].unique()
    data = data[~data['vehicle_id'].isin(ids)]
    new_n = data["vehicle_id"].nunique()
    print(f"\tRemoved {n - new_n} vehicles with time between tests > 2 years")
    n = new_n
    
    # Next remove nay vehicle that has over 100,000 miles per year - outliers
    ids = data.loc[data['mileage_per_year'] > 100_000, 'vehicle_id'].unique()
    data = data[~data['vehicle_id'].isin(ids)]
    new_n = data["vehicle_id"].nunique()
    print(f"\tRemoved {n - new_n} vehicles with mileage per year > 100,000")
    n = new_n
    
    # Sample selection for Transformer model:
    # 1. Get the survival rate distribtion from Figture 3 
    # 2. Sample is then derived from the proportions of vehciles exitting the fleet at each age
    # 3. Use the survial rate to find the maxium Transformer sample size possible, 
    #    given the already selected MOT data sample size
    
    # Survival rate dist:
    fpath = os.path.join(data_path, "survival_rate_data", "figure_3_plot_c_top.csv")
    df_dist = pd.read_csv(fpath, index_col=0)
    df_dist = df_dist[df_dist['fuel'] == veh_name]
    ages = df_dist['age'].to_list()
    surv_rate = df_dist['percentage'].to_list()
    
    # get the proportion between subsequent ages
    surv_rate_diff = []
    for i in range(1, len(surv_rate)):
        surv_rate_diff.append(surv_rate[i-1] - surv_rate[i])
    ages = ages[1:] # not enough data for age == 3
    surv_rate_diff = [s for s in surv_rate_diff if s > 0.0] # remove negative values

    # Find the maxium sample size possible for this survival rate curve and MOT data sample size
    # only using vehciles that left the fleet beteween 2015 and 2021
    first_age = True
    for age, surv in zip(ages, surv_rate_diff):
        n_ids = data.loc[(data['age_year_rounded'] == age) & (data['last_test'] == True) & (data['test_year'].isin([2015, 2016, 2017, 2018, 2019, 2020, 2021])), 'vehicle_id'].nunique()
        
        max_sample_size_for_age = n_ids / surv
        print(f"\tVehicles for Age {age}, Surival Rate {round(surv, 3)}, ID's {n_ids}, Implied Sample Size {int(max_sample_size_for_age)}")
        
        if first_age:
            sample_size = max_sample_size_for_age
            sample_age_limiter = age
            first_age = False
        else:
            if max_sample_size_for_age < sample_size:
                sample_age_limiter = age
            sample_size = min(sample_size, max_sample_size_for_age)
            
    print(f"\tTotal Sample Size: {sample_size}, Limited by Age: {sample_age_limiter}")
    print(f"\tTotal Sample Size: {sample_size} * 2 = {sample_size * 2}") # multiply this by 2 since the limiting age is always much lower (~1/2) than the rest
    
    # Double the sample size since the limiting age is always much lower (~1/2) than the rest
    sample_size *= 2 
    
    print("Selecting Sample for Transformer Model")
    # Now select the vehicles for the sample using sample_size
    ids = []
    for age, surv in zip(ages, surv_rate_diff):
        print(f"\tAge: {age}, Survival Rate: {surv}, Sample Size: {int(sample_size * surv)}")
        ids_age = data.loc[(data['age_year_rounded'] == age) & (data['last_test'] == True) & (data['test_year'].isin([2015, 2016, 2017, 2018, 2019, 2020, 2021])), 'vehicle_id'].unique()
        n_age = int(sample_size * surv)
        try:
            ids += list(np.random.choice(ids_age, size=n_age, replace=False))
        except:
            print(f"\tWarning - number of ids less than sample size {age}, {surv} : {n_age}")
            print(f"\tAdding all ids for this age: {len(ids_age)}")
            ids += list(ids_age)
    data = data[data['vehicle_id'].isin(ids)]
    new_n = data["vehicle_id"].nunique()
    print(f"\tRemoved {n - new_n} vehicles to fit survival rate distribution")
    n = new_n      

    print(f"\tFiltered {veh_name} Sample: {data['vehicle_id'].nunique()}")
    print(f"Memory usage for dataset- {data.memory_usage().sum() / 1e6:.2f} MB")
    print(data.info())
    print(data.describe().to_string())
    
    folder = "Sample_Data"
    filename = f"transformer_training_data_{veh_name}.parquet"
    fpath = os.path.join(data_path, folder, filename)
    data = data.to_parquet(fpath, compression='snappy', engine='pyarrow')
    
    del data
    gc.collect()


In [ ]:
# 6. Prepare Data for Predictions with the transformer model
data_fuels = []
for veh_name in ["car_diesel", "car_petrol", "car_bev", "car_hev"]:
    print(f"Reading in {veh_name} data...")
    directory = "Sample_Data" 
    filename = f"sample_{veh_name}_all_years.parquet"
    file_path = os.path.join(data_path, directory, filename)
    data = pd.read_parquet(file_path, engine='pyarrow')
    
    # Only keep the columns we need for predictions
    categorical_cols = ['fuel_type', 'last_test']
    numerical_cols = ['mileage_per_year', 'test_mileage', 'age_year', 'time_between_tests']#, 'test_mileage_age_indicator', 'mileage_per_year_age_indicator', 'taxi_indicator']
    data_cols = ['vehicle_id', 'test_year', 'first_test', 'make', 'model',
                     'first_use_year', 'fuel efficiency Wh/mi', 'battery capacity (kWh)', 
                     'CO2 g/km', 'mass (kg)']
    predictions_cols = data_cols + categorical_cols + numerical_cols
    data = data[predictions_cols]

    # Add a few filtering steps
    n = data['vehicle_id'].nunique()
    print(f"\tFiltering {veh_name} Data for Training. Initial Sample: {n}")
    
    # remove any ids with NaN for any of the categorical or numerical cols
    ids = data[categorical_cols + numerical_cols + ['vehicle_id']].loc[data[categorical_cols + numerical_cols + ['vehicle_id']].isnull().any(axis=1), 'vehicle_id'].unique()
    data = data[~data['vehicle_id'].isin(ids)]
    new_n = data['vehicle_id'].nunique()
    print(f"\tRemoved {n - new_n} vehicles with NaN values")
    n = new_n
    
    # Next drop any vehicle that has over 2 Years between tests
    ids = data.loc[data['time_between_tests'] > 2, 'vehicle_id'].unique()
    data = data[~data['vehicle_id'].isin(ids)]
    new_n = data['vehicle_id'].nunique()
    print(f"\tRemoved {n - new_n} vehicles with time between tests > 2")
    n = new_n
    
    # Check that all vehicles now have at least 3 records of data - min sequence length is 3
    vehicle_counts = data['vehicle_id'].value_counts()
    ids = vehicle_counts[vehicle_counts < 3].index
    data = data[~data['vehicle_id'].isin(ids)]
    new_n = data['vehicle_id'].nunique()
    print(f"\tRemoved {n - new_n} vehicles with less than 3 records")
    n = new_n
    
    # Next remove nay vehicle that has over 100,000 miles per year
    ids = data.loc[data['mileage_per_year'] > 100_000, 'vehicle_id'].unique()
    data = data[~data['vehicle_id'].isin(ids)]
    new_n = data['vehicle_id'].nunique()
    print(f"\tRemoved {n - new_n} vehicles with mileage per year > 100,000")
    n = new_n
        
    # Sample selection
    ids = []
    # Find the number of vehicle_ids in the fleet for each first_use year
    for year in range(2005, 2021, 1):
        n = data.loc[data['first_use_year'] == year, 'vehicle_id'].nunique()
        print(f"\tFirst Use Year: {year}, Vehicles: {n}")   
        
        # Now select 50_000 from each year, except for car_bev use all ids
        ids_year = data.loc[data['first_use_year'] == year, 'vehicle_id'].unique()
        if veh_name == 'car_bev':
            ids += list(ids_year)
        else:
            try:
                ids += list(np.random.choice(ids_year, size=50_000, replace=False))
            except:
                print(f"\tWarning: with sample size {year} : {50_000}")
                print(f"\tAdding all ids for this year: {len(ids_year)}")
                ids += list(ids_year)
    data = data[data['vehicle_id'].isin(ids)]
    new_n = data['vehicle_id'].nunique()
    print(f"\tFiltered {veh_name} Sample: {new_n}")
    print(f"\tFiltered {veh_name} Sample: {data['vehicle_id'].nunique()}")
    folder = "Sample_Data"
    filename = f"transformer_prediction_data_{veh_name}.parquet"
    file_path = os.path.join(data_path, folder, filename)
    data = data.to_parquet(file_path, compression='snappy', engine='pyarrow')
    